In [1]:
import pandas as pd
import requests
import os

In [2]:
df_2 = pd.read_csv('city_bikes_sample.csv')
df_2

,name,latitude,longitude,free_bikes,id
0,Queen St E / Woodward Ave,43.665269,-79.319796,14,1
1,Primrose Ave / Davenport Rd,43.671420,-79.445947,2,2
2,Queen St. E / Rhodes Ave.,43.666224,-79.317693,13,3
3,Bond St / Queen St E,43.653236,-79.376716,7,4
4,Church St / Alexander St,43.663722,-79.380288,10,5


In [3]:
location = df_2[['latitude', 'longitude']]

location_as_string = location.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
vals = [','.join(ele.split()) for ele in location_as_string]
print(vals)

['43.665269,-79.319796', '43.671420,-79.445947', '43.666224,-79.317693', '43.653236,-79.376716', '43.663722,-79.380288']


In [4]:
api_key = os.environ["FOURSQUARE_API_KEY"]
def result_foursquare(vals):
    responses = []
    for row in vals:
        url = f'https://api.foursquare.com/v3/places/search?ll={row}&radius=1000&fields=rating%2Cname%2Clocation'
        headers = {'Accept': 'application/json',
                   'Authorization': api_key}
        response = requests.request("GET", url, headers=headers)
        if response.status_code == 200:
            responses.append(response.json())
        else:
            print(f"Unable to get coordinates{row}: {response.status_code}")

    return responses

In [5]:
info = result_foursquare(vals)

In [6]:
info

[{'results': [{'location': {'address': '150 Greenwood Ave',
     'country': 'CA',
     'cross_street': 'Dundas St E',
     'formatted_address': '150 Greenwood Ave (Dundas St E), Toronto ON M4L 2R1',
     'locality': 'Toronto',
     'postcode': 'M4L 2R1',
     'region': 'ON'},
    'name': 'Leslieville Farmers Market',
    'rating': 9.1},
   {'location': {'address': '1001 Eastern Ave',
     'country': 'CA',
     'cross_street': 'Woodfield Rd',
     'formatted_address': '1001 Eastern Ave (Woodfield Rd), Toronto ON M4L 1A8',
     'locality': 'Toronto',
     'postcode': 'M4L 1A8',
     'region': 'ON'},
    'name': 'Rorschach Brewing',
    'rating': 8.7},
   {'location': {'address': '1483 Queen St E',
     'country': 'CA',
     'cross_street': '',
     'formatted_address': '1483 Queen St E, Toronto ON M4L 1E2',
     'locality': 'Toronto',
     'postcode': 'M4L 1E2',
     'region': 'ON'},
    'name': 'Chick-N-Joy',
    'rating': 7.5},
   {'location': {'address': '1592 Queen St E',
     'count

In [20]:
flat_results = []
for place in info:
    results = place['results']
    for result in results:
        flat_results.append(result)

In [21]:
flat_results

[{'location': {'address': '150 Greenwood Ave',
   'country': 'CA',
   'cross_street': 'Dundas St E',
   'formatted_address': '150 Greenwood Ave (Dundas St E), Toronto ON M4L 2R1',
   'locality': 'Toronto',
   'postcode': 'M4L 2R1',
   'region': 'ON'},
  'name': 'Leslieville Farmers Market',
  'rating': 9.1},
 {'location': {'address': '1001 Eastern Ave',
   'country': 'CA',
   'cross_street': 'Woodfield Rd',
   'formatted_address': '1001 Eastern Ave (Woodfield Rd), Toronto ON M4L 1A8',
   'locality': 'Toronto',
   'postcode': 'M4L 1A8',
   'region': 'ON'},
  'name': 'Rorschach Brewing',
  'rating': 8.7},
 {'location': {'address': '1483 Queen St E',
   'country': 'CA',
   'cross_street': '',
   'formatted_address': '1483 Queen St E, Toronto ON M4L 1E2',
   'locality': 'Toronto',
   'postcode': 'M4L 1E2',
   'region': 'ON'},
  'name': 'Chick-N-Joy',
  'rating': 7.5},
 {'location': {'address': '1592 Queen St E',
   'country': 'CA',
   'cross_street': '',
   'formatted_address': '1592 Queen

In [33]:
df_4 = pd.json_normalize(info, 'results', [['context', 'geo_bounds', 'circle', 'center', 'latitude'],
                                         ['context', 'geo_bounds', 'circle', 'center', 'longitude']])
    
    

In [38]:
ID = df_4.groupby(['context.geo_bounds.circle.center.latitude',
                           'context.geo_bounds.circle.center.longitude']).ngroup()+1


In [35]:
df_4

,name,rating,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region,location.address_extended,context.geo_bounds.circle.center.latitude,context.geo_bounds.circle.center.longitude,ID
0,Leslieville Farmers Market,9.1,150 Greenwood Ave,CA,Dundas St E,"150 Greenwood Ave (Dundas St E), Toronto ON M4...",Toronto,M4L 2R1,ON,NaN,43.665269,-79.319796,3
1,Rorschach Brewing,8.7,1001 Eastern Ave,CA,Woodfield Rd,"1001 Eastern Ave (Woodfield Rd), Toronto ON M4...",Toronto,M4L 1A8,ON,NaN,43.665269,-79.319796,3
2,Chick-N-Joy,7.5,1483 Queen St E,CA,,"1483 Queen St E, Toronto ON M4L 1E2",Toronto,M4L 1E2,ON,NaN,43.665269,-79.319796,3
3,Velotique,7.8,1592 Queen St E,CA,,"1592 Queen St E, Toronto ON M4L 1G1",Toronto,M4L 1G1,ON,NaN,43.665269,-79.319796,3
4,O Sushi,7.5,6 Coxwell Ave,CA,Queen,"6 Coxwell Ave (Queen), Toronto ON M4L 3A7",Toronto,M4L 3A7,ON,NaN,43.665269,-79.319796,3
5,Woodbine Park,8.5,Queen St,CA,at Kingston Rd,"Queen St (at Kingston Rd), Toronto ON M4L 1G7",Toronto,M4L 1G7,ON,NaN,43.665269,-79.319796,3
6,The Sidekick,8.8,1374 Queen St E,CA,at Greenwood Ave,"1374 Queen St E (at Greenwood Ave), Toronto ON...",Toronto,M4L 1C9,ON,NaN,43.665269,-79.319796,3
7,The Burger's Priest,7.4,1636 Queen St E,CA,btwn Coxwell Ave. & Kingston Rd.,1636 Queen St E (btwn Coxwell Ave. & Kingston ...,Toronto,M4L 1G3,ON,NaN,43.665269,-79.319796,3
8,Casa Di Giorgios,7.4,1646 Queen St E,CA,"at Eastern Ave,","1646 Queen St E (at Eastern Ave,), Toronto ON ...",Toronto,M4L 1G3,ON,NaN,43.665269,-79.319796,3
9,British Style Fish & Chips,7.6,73 Coxwell Ave,CA,Dundas St & Coxwell St,"73 Coxwell Ave (Dundas St & Coxwell St), Toron...",Toronto,M4L 3B1,ON,NaN,43.665269,-79.319796,3


In [42]:
all_data = []
for id_, d in enumerate(info, 1):
    for r in d["results"]:
        all_data.append({"ID": id_, "name": r["name"], "rating": r["rating"]})

df = pd.DataFrame(all_data)
df

,ID,name,rating
0,1,Leslieville Farmers Market,9.1
1,1,Rorschach Brewing,8.7
2,1,Chick-N-Joy,7.5
3,1,Velotique,7.8
4,1,O Sushi,7.5
5,1,Woodbine Park,8.5
6,1,The Sidekick,8.8
7,1,The Burger's Priest,7.4
8,1,Casa Di Giorgios,7.4
9,1,British Style Fish & Chips,7.6


In [39]:
df_4 = pd.DataFrame({
                  'id': ID,
                  'name' : names,
                  'rating' : ratings,
                  'location' : locations
                  
                  })

In [40]:
df_4

,id,name,rating,location
0,3,Leslieville Farmers Market,9.1,150 Greenwood Ave
1,3,Rorschach Brewing,8.7,1001 Eastern Ave
2,3,Chick-N-Joy,7.5,1483 Queen St E
3,3,Velotique,7.8,1592 Queen St E
4,3,O Sushi,7.5,6 Coxwell Ave
5,3,Woodbine Park,8.5,Queen St
6,3,The Sidekick,8.8,1374 Queen St E
7,3,The Burger's Priest,7.4,1636 Queen St E
8,3,Casa Di Giorgios,7.4,1646 Queen St E
9,3,British Style Fish & Chips,7.6,73 Coxwell Ave


In [40]:
id

5

In [38]:
ids = [result['id'] for result in info]

KeyError: 'id'

In [26]:

locations = [result['location'].get('address') for result in flat_results]
ratings = [result.get('rating') for result in flat_results]
names = [result['name'] for result in flat_results]

In [ ]:
df_3 = pd.json_normalize(data, 'results', [['context', 'geo_bounds', 'circle', 'center', 'latitude'],
                                         ['context', 'geo_bounds', 'circle', 'center', 'longitude']])

In [49]:
df_3 = pd.DataFrame({
                  'id': ID,
                  'name' : names,
                  'rating' : ratings,
                  'location' : locations
                  
                  })

In [50]:
df_3

,id,name,rating,location
0,3,Leslieville Farmers Market,9.1,150 Greenwood Ave
1,3,Rorschach Brewing,8.7,1001 Eastern Ave
2,3,Chick-N-Joy,7.6,1483 Queen St E
3,3,Velotique,7.4,1592 Queen St E
4,3,O Sushi,7.5,6 Coxwell Ave
5,3,Woodbine Park,8.5,Queen St
6,3,The Sidekick,8.8,1374 Queen St E
7,3,The Burger's Priest,7.6,1636 Queen St E
8,3,Casa Di Giorgios,7.4,1646 Queen St E
9,3,British Style Fish & Chips,7.6,73 Coxwell Ave
